In [404]:
#pip install chromadb

In [405]:
import os
import pandas as pd
from datetime import datetime
import numpy as np
import nltk
import torch
from io import BytesIO

from tqdm.notebook import tqdm
import chromadb

import pandas as pd
import torch

from transformers import BertTokenizer, BertModel
from transformers import AutoModel, AutoTokenizer



# Ensure that the Punkt Tokenizer Models are downloaded
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\johnny\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [406]:
#download_dir = 'D:\\downloads'
download_dir = 'D:\\downloads\\amazon_customer_reviews'
print(download_dir)

D:\downloads\amazon_customer_reviews


In [407]:
#df_pickle_paragraphs_filename = os.path.join(download_dir,"amazon_reviews_pickle_paragraphs_20240129.pkl")
df_pickle_paragraphs_filename = os.path.join(download_dir,"amazon_reviews_pickle_paragraphs_20240219.pkl")
df_paragraph = pd.read_pickle(df_pickle_paragraphs_filename)

FileNotFoundError: [Errno 2] No such file or directory: 'D:\\downloads\\amazon_customer_reviews\\amazon_reviews_pickle_paragraphs_20240219.pkl'

In [ ]:
df_paragraph.columns

In [ ]:
df_paragraph.shape

In [ ]:
df_paragraph.iloc[5]

In [ ]:
#df_pickle_sentences_filename = os.path.join(download_dir,"amazon_reviews_pickle_sentences_20240129.pkl")
df_pickle_sentences_filename = os.path.join(download_dir,"amazon_reviews_pickle_sentences_20240219.pkl")

df_sentences = pd.read_pickle(df_pickle_sentences_filename)

In [ ]:
df_sentences.columns

In [ ]:
df_sentences.shape

In [ ]:
df_sentences.iloc[5]

In [ ]:
df_sentences.iloc[5]['Summary_vector'].shape

In [ ]:
df_sentences.iloc[5]['Summary_vector'][0].shape

In [ ]:
type(df_sentences.iloc[5]['Summary_vector'][0])

In [ ]:
df_sentences.columns

In [ ]:
df = df_sentences.iloc[0:2].copy()

import numpy as np
import pandas as pd

# Example DataFrame
# df = pd.DataFrame({'Summary_vector': [... your data ...]})

# Process the 'Summary_vector' column
def process_vector(v):
    # Convert to numpy array with dtype float32
    v_array = np.array(v, dtype=np.float32)
    
    # Optionally reshape to 2D (1, -1) if required by your application
    # v_array = v_array.reshape(1, -1)
    
    # Return processed vector
    return v_array

# Apply processing to each vector in the column
df['Processed_vector'] = df['Summary_vector'].apply(process_vector)

# Check the result for the first few rows
print(df['Processed_vector'].head())


import numpy as np

def preprocess_vector(v):
    return np.array(v, dtype=np.float32).flatten()

# Apply preprocessing to each vector column
df_sentences['Summary_vector'] = df_sentences['Summary_vector'].apply(preprocess_vector)
df_sentences['Sentence_vector'] = df_sentences['Sentence_vector'].apply(preprocess_vector)
df_paragraph['Paragraph_vector'] = df_paragraph['Paragraph_vector'].apply(preprocess_vector)
# Repeat for the third vector column if exists


In [ ]:
import faiss

def create_faiss_index_old(vectors):
    dimension = vectors.shape[1]
    index = faiss.IndexFlatL2(dimension)
    index.add(vectors)
    return index

In [ ]:
def create_faiss_index_old(vectors):
    # Example setup for FAISS index - this is conceptual
    index = faiss.IndexFlatL2(vectors.shape[1])
    # Example of batch addition with tqdm progress bar
    for i in tqdm(range(0, vectors.shape[0], 1000), desc="Adding vectors to FAISS index"):
        index.add(vectors[i:i+1000])
    return index


In [ ]:
len(df_sentences.iloc[0]['Summary_vector'])

In [ ]:
len(df_sentences.iloc[167494]['Summary_vector'])

In [ ]:
df_paragraph.columns

In [ ]:
df_sentences.iloc[167494]

In [ ]:
df_paragraph = df_paragraph.dropna(subset=['Paragraph', 'Summary'])

In [ ]:
print(f"Original df_sentences rows: {len(df_sentences)}")
# Drop rows where 'Summary' is NaN
df_sentences= df_sentences.dropna(subset=['Summary'])

# Optionally, also drop rows where 'Sentence' is NaN if relevant to your analysis
df_sentences = df_sentences.dropna(subset=['Sentence'])

print(f"Cleaned df_sentences rows: {len(df_sentences)}")


In [ ]:
from tqdm.auto import tqdm
import faiss
import numpy as np
import pandas as pd

def create_faiss_index(vectors, n_clusters):
    dimension = vectors.shape[1]  # Dimension of the vectors
    kmeans = faiss.Kmeans(dimension, n_clusters, niter=20, verbose=True, spherical=False)
    kmeans.train(vectors)
    return kmeans

def cluster_vectors_and_save(df, vector_column, n_clusters, cluster_col_name, distance_col_name):
    # Filter out NaNs and prepare the vectors
    filtered_df = df.dropna(subset=[vector_column])
    vectors = np.vstack(filtered_df[vector_column].apply(np.array).to_list())

    # Initialize the progress bar
    tqdm.pandas(desc=f"Clustering {vector_column}")
    
    # Create a FAISS KMeans index for clustering
    print(f"Training KMeans for {vector_column}...")
    kmeans = create_faiss_index(vectors, n_clusters)

    # Assign vectors to the nearest cluster and capture distances
    print(f"Assigning clusters for {vector_column}...")
    distances, cluster_assignments = kmeans.index.search(vectors, 1)

    # Since we filtered NaNs, align results with the original DataFrame indices
    valid_indices = filtered_df.index

    # Update the original DataFrame
    df.loc[valid_indices, cluster_col_name] = cluster_assignments.flatten()
    df.loc[valid_indices, distance_col_name] = distances.flatten()

    return df, kmeans

# Example usage with a specified number of clusters
n_clusters = 100

# Assuming df_sentences and df_paragraph are your DataFrames and they are properly initialized
df_sentences, index_summary  = cluster_vectors_and_save(df_sentences, 'Summary_vector', n_clusters, 'Summary_Cluster', 'Summary_Distance')
df_sentences, index_sentence  = cluster_vectors_and_save(df_sentences, 'Sentence_vector', n_clusters, 'Sentence_Cluster', 'Sentence_Distance')
df_paragraph, index_paragraph  = cluster_vectors_and_save(df_paragraph, 'Paragraph_vector', n_clusters, 'Paragraph_Cluster', 'Paragraph_Distance')


In [ ]:
df_sentences.columns

In [ ]:
df_sentences.iloc[0]

In [ ]:
df_paragraph.columns

In [ ]:
df_paragraph.iloc[1]


In [ ]:
cluster = 12
cluster_0_df = df_paragraph[df_paragraph['Paragraph_Cluster'] == cluster]


In [ ]:
#cluster_0_df

In [ ]:
# Count the number of rows in each cluster for paragraph vectors
paragraph_cluster_counts = df_paragraph['Paragraph_Cluster'].value_counts().sort_index()
#print("Paragraph Cluster Distribution:\n", paragraph_cluster_counts)


In [ ]:
# Count the number of rows in each cluster for summary vectors
summary_cluster_counts = df_sentences['Summary_Cluster'].value_counts().sort_index()
#print("Summary Cluster Distribution:\n", summary_cluster_counts)


In [ ]:
# Count the number of rows in each cluster for sentence vectors
sentence_cluster_counts = df_sentences['Sentence_Cluster'].value_counts().sort_index()
#print("Sentence Cluster Distribution:\n", sentence_cluster_counts)


In [ ]:
import matplotlib.pyplot as plt

# Plotting the distribution of paragraph clusters
paragraph_cluster_counts.plot(kind='bar', figsize=(10, 6))
plt.title('Distribution of Paragraph Clusters')
plt.xlabel('Cluster ID')
plt.ylabel('Number of Rows')
plt.show()


In [ ]:
# Optionally, print some sample paragraphs to get a feel for the content
for paragraph in cluster_0_df['Paragraph'].sample(n=5).values:
    print(paragraph, "\n")


In [ ]:
distance_stats = cluster_0_df['Paragraph_Distance'].describe()
print(distance_stats)


In [ ]:
#pip install wordcloud

In [ ]:
from wordcloud import WordCloud

# Concatenate all paragraphs in the cluster into a single string
text = " ".join(paragraph for paragraph in cluster_0_df['Paragraph'])

# Generate a word cloud image
wordcloud = WordCloud(background_color="white").generate(text)

# Display the generated image:
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()



In [ ]:
type(wordcloud)

In [ ]:
#dir(wordcloud)


In [ ]:
print("Word frequencies:", wordcloud.words_)

In [ ]:
#print("Word layout:", wordcloud.layout_)

# Latent Dirichlet Allocation (LDA)  

In [ ]:
cluster_0_df = df_paragraph[df_paragraph['Paragraph_Cluster'] == cluster]
texts = cluster_0_df['Paragraph'].values


In [ ]:
from gensim import corpora
from gensim.models.ldamodel import LdaModel
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer

# Ensure you have the necessary NLTK data
import nltk
nltk.download('stopwords')

# Tokenization and stop words removal
tokenizer = RegexpTokenizer(r'\w+')
stop_words = set(stopwords.words('english'))
texts_tokenized = [[word for word in tokenizer.tokenize(text.lower()) if word not in stop_words] for text in texts]

# Creating a dictionary representation of the documents
dictionary = corpora.Dictionary(texts_tokenized)

# Filtering out extremes to limit the number of features
dictionary.filter_extremes(no_below=5, no_above=0.5)

# Creating the document-term matrix
corpus = [dictionary.doc2bow(text) for text in texts_tokenized]


# LDA

In [ ]:
num_topics = 50  # Adjust based on your dataset and needs
ldamodel = LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=15)

# Print the topics
#topics = ldamodel.print_topics(num_words=4)
#for topic in topics:
#    print(topic)
# Print all 50 topics with the top 4 words in each
all_topics = ldamodel.print_topics(num_topics=50, num_words=4)
for topic in all_topics:
    print(topic)

In [ ]:
stop

In [ ]:
stop

In [ ]:
# way slow
from tqdm.auto import tqdm

def find_top_similarities_and_distances(index, vectors, k=21):
    """
    Find the top k-1 most similar items and their distances for each vector in the given index.
    Exclude the first result to skip the query item itself.
    """
    similar_indices = []
    distances_list = []
    
    # Perform the search for each vector
    for i in tqdm(range(vectors.shape[0]), desc='Finding Similarities and Distances'):
        D, I = index.search(vectors[i:i+1], k)  # Search for the top k similar items
        similar_indices.append(I[0][1:])  # Skip the first result and keep the next 20
        distances_list.append(D[0][1:])  # Skip the distance to itself and keep the next 20
    
    return similar_indices, distances_list


In [ ]:
# way slow 128 days each
# Example for the summary vectors and index
top20_summary_indices, top20_summary_distances = find_top_similarities_and_distances(index_summary, summary_vectors, k=21)

# Update the DataFrame
df_sentences['Top20_Summary_Indices'] = top20_summary_indices
df_sentences['Top20_Summary_Distances'] = top20_summary_distances

# Repeat for sentence and paragraph vectors and indexes
top20_sentence_indices, top20_sentence_distances = find_top_similarities_and_distances(index_sentence, sentence_vectors, k=21)
df_sentences['Top20_Sentence_Indices'] = top20_sentence_indices
df_sentences['Top20_Sentence_Distances'] = top20_sentence_distances

top20_paragraph_indices, top20_paragraph_distances = find_top_similarities_and_distances(index_paragraph, paragraph_vectors, k=21)
df_paragraph['Top20_Paragraph_Indices'] = top20_paragraph_indices
df_paragraph['Top20_Paragraph_Distances'] = top20_paragraph_distances


In [ ]:
stop # some of the code below duplicates the code in the above block

In [ ]:
df_pickle_sentences_filename_clusters = os.path.join(download_dir,"amazon_reviews_pickle_sentences_20240129_clusters.pkl")

df_sentences.to_pickle(df_pickle_sentences_filename_clusters)  # Python pickle format


In [ ]:
df_pickle_paragraph_filename_clusters = os.path.join(download_dir,"amazon_reviews_pickle_paragraph_20240129_clusters.pkl")

df_paragraph.to_pickle(df_pickle_paragraph_filename_clusters)  # Python pickle format

In [ ]:
df = df_sentences.iloc[0:2]

In [ ]:
df

In [ ]:
df.iloc[0]['Summary_vector']

In [ ]:
import faiss
import numpy as np

# Retrieve vectors from your DataFrame
vector1 = df.iloc[0]['Processed_vector']
vector2 = df.iloc[1]['Processed_vector']

# Ensure both vectors are numpy arrays and have the same shape
vector1 = np.array(vector1, dtype=np.float32).reshape(1, -1)
vector2 = np.array(vector2, dtype=np.float32).reshape(1, -1)

# Verify shapes are correct (this step is just for demonstration and can be removed later)
print(f"Vector1 Shape: {vector1.shape}")
print(f"Vector2 Shape: {vector2.shape}")

# Combine vectors into a single 2D array
vectors = np.vstack((vector1, vector2))

# Continue with FAISS index creation and querying
dimension = vectors.shape[1]  # Dimension of the vectors
index = faiss.IndexFlatL2(dimension)  # L2 distance for similarity measure
index.add(vectors)  # Add vectors to the index

# Perform a query with the first vector
query_vector = np.array([vector1], dtype=np.float32).reshape(1, -1)
k = 2  # Find the 2 nearest neighbors, including the query vector itself
distances, indices = index.search(query_vector, k)

# Display the results
print("Distances:", distances)
print("Indices:", indices)


In [ ]:
import faiss
import numpy as np

# Step 1: Prepare the vectors
# Assuming vector1 is the one you provided and vector2 is similarly obtained
vector1 = [1.07387170e-01,  1.54322982e-02, ...]  # Truncated for brevity
vector2 = [ ... ]  # Similar structure to vector1
vector1 = df.iloc[0]['Summary_vector']
vector2 = df.iloc[1]['Summary_vector']

vectors = np.array([vector1, vector2], dtype=np.float32)

# Step 2: Create a FAISS index
dimension = vectors.shape[1]  # Dimension of the vectors
index = faiss.IndexFlatL2(dimension)  # L2 distance for similarity measure

# Step 3: Add the vectors to the index
index.add(vectors)

# Step 4: Perform a query
# Use the first vector as a query to find its nearest neighbor
# Note: Including itself, so the nearest neighbor will be itself for k=1
query_vector = np.array([vector1], dtype=np.float32)
k = 2  # Find the 2 nearest neighbors, including the query vector itself
distances, indices = index.search(query_vector, k)

# Display the results
print("Distances:", distances)
print("Indices:", indices)


In [ ]:
df_experiment = df_sentences.head(1000).copy()  # Assuming 'df' is your original DataFrame


In [ ]:
import numpy as np

def preprocess_vectors_for_faiss(vector):
    # Ensure the vector is a numpy array of type float32 and is 2D
    if isinstance(vector, list):  # If the vector is stored as a list
        vector = np.array(vector, dtype=np.float32)
    if vector.ndim == 1:
        vector = vector.reshape(1, -1)
    return vector

# Apply this function to the 'Summary_vector' column
df_experiment['Summary_vector'] = df_experiment['Summary_vector'].apply(preprocess_vectors_for_faiss)


In [ ]:
import faiss

# Assuming you have preprocessed the vectors as 2D arrays
# Flatten the arrays for FAISS index, as it expects flat arrays
vectors = np.vstack(df_experiment['Summary_vector'].to_numpy())

# Create a FAISS index
dimension = vectors.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(vectors.astype(np.float32))  # Ensure data is float32

# Query the index (example)
query_vector = vectors[0:1]  # Use the first vector as a query example
k = 5  # Number of nearest neighbors to find
distances, indices = index.search(query_vector, k)

print(distances, indices)


In [ ]:
type(index)

In [ ]:
print(index)


In [ ]:
n_vectors = index.ntotal
vector_dim = index.d
print(f"Number of vectors: {n_vectors}, Dimensionality: {vector_dim}")


In [ ]:
is_trained = index.is_trained
print(f"Is the index trained? {is_trained}")


In [ ]:
if isinstance(index, faiss.IndexIVFFlat):
    print(f"Number of list: {index.nlist}")
    print(f"Number of probes: {index.nprobe}")


In [ ]:
# Save the index to disk
index_file = "my_index.faiss"
faiss.write_index(index, index_file)

# Load the index (for inspection or use in another program)
index_loaded = faiss.read_index(index_file)


In [ ]:
from transformers import BertTokenizer, BertModel
import torch
import numpy as np

# Initialize MinIO client and BERT model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

def generate_vectors(text):
    # Check if GPU is available and use it; otherwise, use CPU
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    # Send model to device (GPU or CPU)
    model.to(device)
    model.eval()  # Ensure the model is in evaluation mode

    # Ensure no gradient calculations
    with torch.no_grad():
        # Prepare inputs and send them to the device
        inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512).to(device)
        
        # Forward pass, send model outputs back to CPU
        outputs = model(**inputs).last_hidden_state.mean(dim=1).squeeze().to('cpu').numpy()

    # Convert the output to float32 for compatibility and ensure it's flat
    return outputs.astype(np.float32)

# Example usage with a DataFrame
# df_sentence['Summary_vector'] = df_sentence['Summary'].progress_apply(lambda x: generate_vectors(x) if isinstance(x, str) else np.nan)


In [ ]:
vectors = np.stack(df_sentences['Summary_vector'].values).astype(np.float32)

In [ ]:
vectors = df_sentences['Summary_vector'].values

In [ ]:
type(vectors[0])

In [ ]:
vectors = df_sentences['Summary_vector'].values.astype(np.float32)  # Assuming vectors are stored as numpy arrays


In [ ]:
type(vectors)

In [ ]:
stop

In [ ]:
import chromadb

# Initialize the Chroma client with a specified database path
db_path = download_dir  # Specify the path where you want to store the ChromaDB
#chroma_client = chromadb.Client(path=db_path) # this gave an error
#client = chromadb.PersistentClient(path="/path/to/save/to") # from web site
chroma_client = chromadb.PersistentClient(path=db_path)

In [ ]:
def separate_metadata_and_vectors(df):
    vector_columns = [col for col in df.columns if 'vector' in col.lower()]
    metadata_columns = [col for col in df.columns if 'vector' not in col.lower()]
    return metadata_columns, vector_columns


In [ ]:
def load_df_to_chromadb(df, collection_name, text_columns, embedding_columns, db_path, ignore_columns=None):
    if ignore_columns is None:
        ignore_columns = []

    chroma_client = chromadb.PersistentClient(path=db_path)

    try:
        collection = chroma_client.get_collection(collection_name)
    except Exception as e:
        print(f"Collection {collection_name} not found, creating a new one. Error: {e}")
        collection = chroma_client.create_collection(name=collection_name)

    #for _, row in df.iterrows():
    for _, row in tqdm(df.iterrows(), total=len(df), desc=f"Loading to {collection_name}"):
        metadata = {col: row[col] for col in row.index if col not in embedding_columns + text_columns + ignore_columns}
        #embeddings = [row[col] for col in embedding_columns if col in row and col not in ignore_columns]

        # Ensure embeddings are converted to list if they are numpy arrays
        embeddings = [row[col].tolist() for col in embedding_columns if col in row and col not in ignore_columns and isinstance(row[col], np.ndarray)][0]
        embeddings = embeddings[0] if embeddings else None
        if embeddings is not None:
            embeddings_list.append(embeddings)
        
        documents = [row[col] for col in text_columns if col in row]
        
        document_id = f"{collection_name}_{row['Id']}"
        if 'P_index' in row and 'S_sentence_number' in row:
            document_id += f"_{row['P_index']}_{row['S_sentence_number']}"

        collection.add(
            documents=documents,
            embeddings=embeddings,  # Each embedding is added separately
            metadatas=[metadata],
            ids=[document_id]
        )

In [ ]:
def delete_collection(collection):
    try:
        chroma_client.delete_collection(collection)
        print("Collection '{collection}' deleted successfully.")
    except Exception as e:
        print(f"Failed to delete collection: {e}")

In [ ]:
delete_collection('sentences_collection')

In [ ]:
# Create collections

delete_collection('sentences_collection')
sentences_collection = chroma_client.create_collection(name="sentences_collection")


delete_collection('paragraphs_collection')
paragraphs_collection = chroma_client.create_collection(name="paragraphs_collection")


delete_collection('paragraphs_collection')
summary_collection = chroma_client.create_collection(name="paragraphs_collection")


In [ ]:
#type(df_sentences.iloc[0]['Sentence_vector'].tolist())

In [ ]:
#df_sentences.iloc[0]['Sentence_vector'][0].tolist()